In [ ]:
import xarray as xr 
from metpy.units import units
import metpy.constants as constants
import metpy.calc as mpcalc
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
import cmaps
import warnings
import sys
sys.path.append('/public/home/songqh/Documents/python_packages/')
from sqh_toolbox import plot_figure
warnings.filterwarnings("ignore") 
plt.rcParams["font.family"] = "Liberation Sans"
plt.rcParams["font.weight"] = "regular"

In [ ]:
f_u = xr.open_dataset("/public/home/songqh/my_data/new_ERA5/monthly/ERA5_uwnd_1979-2024.nc")
u6 = f_u.u.loc[f_u.time.dt.year.isin(np.arange(1979,2025)) & f_u.time.dt.month.isin(6),1000:100].values[:]
u7 = f_u.u.loc[f_u.time.dt.year.isin(np.arange(1979,2025)) & f_u.time.dt.month.isin(7),1000:100].values[:]
u8 = f_u.u.loc[f_u.time.dt.year.isin(np.arange(1979,2025)) & f_u.time.dt.month.isin(8),1000:100].values[:]
f_v = xr.open_dataset("/public/home/songqh/my_data/new_ERA5/monthly/ERA5_vwnd_1979-2024.nc")
v6 = f_v.v.loc[f_v.time.dt.year.isin(np.arange(1979,2025)) & f_v.time.dt.month.isin(6),1000:100].values[:]
v7 = f_v.v.loc[f_v.time.dt.year.isin(np.arange(1979,2025)) & f_v.time.dt.month.isin(7),1000:100].values[:]
v8 = f_v.v.loc[f_v.time.dt.year.isin(np.arange(1979,2025)) & f_v.time.dt.month.isin(8),1000:100].values[:]
f_w = xr.open_dataset("/public/home/songqh/my_data/new_ERA5/monthly/ERA5_omega_1979-2024.nc")
w6 = f_w.w.loc[f_w.time.dt.year.isin(np.arange(1979,2025)) & f_w.time.dt.month.isin(6),1000:100].values[:]
w7 = f_w.w.loc[f_w.time.dt.year.isin(np.arange(1979,2025)) & f_w.time.dt.month.isin(7),1000:100].values[:]
w8 = f_w.w.loc[f_w.time.dt.year.isin(np.arange(1979,2025)) & f_w.time.dt.month.isin(8),1000:100].values[:]
f_t = xr.open_dataset("/public/home/songqh/my_data/new_ERA5/monthly/ERA5_temperature_1979-2024.nc")
t5 = f_t.t.loc[f_t.time.dt.year.isin(np.arange(1979,2025)) & f_t.time.dt.month.isin(5),1000:100].values[:]
t6 = f_t.t.loc[f_t.time.dt.year.isin(np.arange(1979,2025)) & f_t.time.dt.month.isin(6),1000:100].values[:]
t7 = f_t.t.loc[f_t.time.dt.year.isin(np.arange(1979,2025)) & f_t.time.dt.month.isin(7),1000:100].values[:]
t8 = f_t.t.loc[f_t.time.dt.year.isin(np.arange(1979,2025)) & f_t.time.dt.month.isin(8),1000:100].values[:]

In [ ]:
lon = f_u['longitude'].values
lat = f_u['latitude'].values
lev = f_u['level'].loc[1000:100].values

In [ ]:
time6 = 30*24*60*60*units.s
time7 = 31*24*60*60*units.s
time8 = 31*24*60*60*units.s
dx, dy = mpcalc.lat_lon_grid_deltas(lon, lat)

In [ ]:
dTdt6 = t6*units.K - t5*units.K
dTdt7 = t7*units.K - t6*units.K
dTdt8 = t8*units.K - t7*units.K
temadvect6 = np.zeros((46,27,181,360))
temadvect7 = np.zeros((46,27,181,360))
temadvect8 = np.zeros((46,27,181,360))
for y in range(46):
    for i in range(27):
        temadvect6[y,i,:,:] = mpcalc.advection(t6[y,i], u6[y,i], v6[y,i],dx=dx, dy=dy,x_dim=-1, y_dim=-2)
        temadvect7[y,i,:,:] = mpcalc.advection(t7[y,i], u7[y,i], v7[y,i],dx=dx, dy=dy,x_dim=-1, y_dim=-2)
        temadvect8[y,i,:,:] = mpcalc.advection(t8[y,i], u8[y,i], v8[y,i],dx=dx, dy=dy,x_dim=-1, y_dim=-2)
temadvect6 = temadvect6 * units.K/units.s
temadvect7 = temadvect7 * units.K/units.s
temadvect8 = temadvect8 * units.K/units.s

In [ ]:
pressure = np.array(lev).reshape((1,27,1,1))*units.hPa
t_k6 = t6*units.K
ss6 = (constants.dry_air_gas_constant* t_k6)/constants.dry_air_spec_heat_press/pressure - np.gradient(t_k6,axis=1)/np.gradient(pressure,axis=1)
ver6 = w6*(units.Pa/units.s)*ss6
t_k7 = t7*units.K
ss7 = (constants.dry_air_gas_constant* t_k7)/constants.dry_air_spec_heat_press/pressure - np.gradient(t_k7,axis=1)/np.gradient(pressure,axis=1)
ver7 = w7*(units.Pa/units.s)*ss7
t_k8 = t8*units.K
ss8 = (constants.dry_air_gas_constant* t_k8)/constants.dry_air_spec_heat_press/pressure - np.gradient(t_k8,axis=1)/np.gradient(pressure,axis=1)
ver8 = w8*(units.Pa/units.s)*ss8


In [ ]:
q6 = ((dTdt6 - temadvect6*time6 - ver6*time6)*constants.dry_air_spec_heat_press)/time6
q7 = ((dTdt7 - temadvect7*time7 - ver7*time7)*constants.dry_air_spec_heat_press)/time7
q8 = ((dTdt8 - temadvect8*time8 - ver8*time8)*constants.dry_air_spec_heat_press)/time8

In [ ]:
integral_q6 = np.trapz(q6[:,:],lev[::-1],axis=1)
integral_q7 = np.trapz(q7[:,:],lev[::-1],axis=1)
integral_q8 = np.trapz(q8[:,:],lev[::-1],axis=1)
integral_q = (integral_q6 + integral_q7 + integral_q8) / 3
q_mean = np.array(((q6 + q7 + q8) / 3))

In [ ]:
np.save("/public/home/songqh/project/ISM_impact_Antarctica/data/integral_q.npy",np.array(integral_q))
np.save("/public/home/songqh/project/ISM_impact_Antarctica/data/q_mean.npy",q_mean)

In [ ]:
cycle_mean,cycle_lon = add_cyclic_point(integral_q.mean(0), coord=lon)
fig = plt.figure(figsize=(12,5))
ax1 = fig.add_subplot(111,projection = ccrs.PlateCarree(central_longitude=120))
ax1.draw_map([0,360,-20,40],lonspec=60,latspec=20)
ax1.set_title("JJA mean <Q1>",fontsize=24,loc='center')
rect = plt.Rectangle((70, 10), 30, 18, edgecolor='g',zorder=10,
                     facecolor='None',linewidth=2,transform=ccrs.PlateCarree())
ax1.add_patch(rect)
cf1 = ax1.contourf(cycle_lon,lat,cycle_mean, 
                   zorder=2,levels=np.linspace(-60,60,11),
                   extend = 'both',transform=ccrs.PlateCarree(), 
                   cmap=cmaps.BlueDarkRed18)
cb = fig.colorbar(cf1,orientation='vertical',shrink=0.55,pad=0.03)
cb.ax.set_colorbar()